# MySQL 연결

+ 일반 사용자와 DB를 파이썬 프로그램으로 연결: 일반 사용자  - (파이썬) - (Library 프로그램) -  DB

In [1]:
# pip install mysqlclient: Library 프로그램 설치

!pip install mysqlclient

In [2]:
import MySQLdb

In [5]:
config={"host":"127.0.0.1", "user":"root", "password":"1111", "database":"testdb"}  # testdb 접속.
conn = MySQLdb.connect(**config)  # 가변인수로 받아 사용
print(conn)  # 접속 확인(주소값)

conn.close()  # 사용 후 반드시 종료 필요

<_mysql.connection open to '127.0.0.1' at 00000000046D77B8>


### SELECT 관련

In [45]:

conn = MySQLdb.connect(**config)
cur = conn.cursor()  # cursor(): 데이터 조작 위한 위치로.
                     # cur 변수에서 모든 작업 가능
print(cur)  # 접속 확인(주소값)

sql = "select empno, ename, sal, job, deptno from scott_emp"  # 명령어 준비
cur.execute(sql)  # execute(): 현재 명령어를 db에 전달하여 결과값 받아와 cur 객체에 저장

# for data in cur:  # data가 여러 개이므로 반복문 사용
#     print(data)  # tuple로 가져온다
#     print("%d, %s, %d, %s, %d" %data)  # data 하나씩 가져온다

for (empno, ename, sal, job, deptno) in cur:
    print(empno, ename, sal, job, deptno)  # data 가져오는 다른 방법
    
conn.close()

7369 SMITH 800.0 CLERK 20
7499 ALLEN 1600.0 SALESMAN 30
7521 WARD 1250.0 SALESMAN 30
7566 JONES 2975.0 MANAGER 20
7654 MARTIN 1250.0 SALESMAN 30
7698 BLAKE 2850.0 MANAGER 30
7782 CLARK 2450.0 MANAGER 10
7788 SCOTT 3000.0 ANALYST 20
7839 KING 5000.0 PRESIDENT 10
7844 TURNER 1500.0 SALESMAN 30
7876 ADAMS 1100.0 CLERK 20
7900 JAMES 950.0 CLERK 30
7902 FORD 3000.0 ANALYST 20
7934 MILLER 1300.0 CLERK 10


### INSERT 관련

In [17]:
# values 값 직접 넣기

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = """insert into scott_emp(empno, ename, sal, job, mgr, deptno, hiredate) 
            values(1, 'Tom', 7000, 'clerk', 7788, 20, now())"""

cur.execute(sql)  # DML 작업(insert, update, delete)는 항상 끝에 conn.commit() 사용
conn.commit()

conn.close()

In [25]:
# values 값들의 변수 처리

conn = MySQLdb.connect(**config)
cur = conn.cursor()

v_empno = 2
v_ename = 'Jane'
v_sal = 1000

sql = "insert into scott_emp(empno, ename, sal) values(" + str(v_empno) + ", '" + v_ename + "', " \
        + str(v_sal) + ")"
  # 변수 이용하여 values 값 처리. 변수값 입력 위해 ""로 문자 입력 끊고 +로 연결(구분 처리 중요)
print(sql)  # Query 이상 없는지 확인 필요. v_ename 문자열 인식 위해 '' 추가

cur.execute(sql) 
conn.commit()

conn.close()

# Query문에 직접 입력: 불편 + 보안취약(SQL Injection 해킹 공격에 취약)

In [28]:
# sql 코드의 단순화: 값의 자리 마련 후 따로 전달

conn = MySQLdb.connect(**config)
cur = conn.cursor()

v_empno = 3
v_ename = 'John'
v_sal = 600

sql = "insert into scott_emp(empno, ename, sal) values(%s, %s, %s)"  # 값 자리 마련

sql_data = (str(v_empno), str(v_ename), str(v_sal))
cur.execute(sql, sql_data)  # 변수 따로 선언하여 values 값에 전달

conn.commit()

conn.close()

### DELETE 관련: INSERT와 동일

In [31]:

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "delete from scott_emp where empno between 1 and 3"

cur.execute(sql)
conn.commit()

conn.close()

## MySQL 활용 예제

+ 파일 형식의 데이터를 받아 DB에 저장

+ DB 접속하여 테이블 작성

    CREATE TABLE IF NOT EXISTS suppliers(
        Supplier_Name varchar(20),
        Invoice_Number varchar(20),
        Part_Number varchar(20),
        Cost float,
        Purchase_Date date
    );
    
  cf. IF NOT EXISTS: (테이블을 만들 때)데이터의 값이 있으면 업데이트, 없으면 새로 만들어라

In [38]:
import csv  # csv 파일 전문적으로 다루는 모듈. 작업 편이성 위해 import
import datetime  # 날짜 전문적으로 다루는 모듈. 작업 편이성 위해 import
from datetime import datetime  # 파일 이름과 클래스명 같아 축소 사용 위해
                                # datetime 모듈로부터 datetime 클래스를 가져온다

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "delete from suppliers"  # 기존 내용 삭제

cur.execute(sql)
conn.commit()

conn.close()

In [39]:

conn = MySQLdb.connect(**config)
cur = conn.cursor()

file = csv.reader(open("data/input.csv", "r"))  # reader(): 기존 함수 기능 + 새로운 기능 추가 사용
                                                # 파이썬 기본 함수인 open()에 기능을 추가해서 사용하기 위함
#print(file)  확인

header = next(file)  # file의 한 줄을 불러와(next(file)) 제목으로 저장
#print(header)  확인

for row in file:
    #print(row)  확인. DB 전달 용이성 위해 데이터 다듬기 필요(문자형으로 입력된 날짜를 날짜형식으로 변환)
    data = []
    for col in range(len(header)):  # header의 개수만큼 반복(column의 개수 만큼 = 5번)
        if col == 4:  # header의 5번째 위치(날짜값)에 온다면
            tmp = datetime.strptime(row[col], "%Y-%m-%d")  # strptime(대상, 양식): 문자형 날짜를 날짜형으로 변환시키는 함수
            data.append(tmp)
        else:
            data.append(row[col])  # 아니라면 그대로 출력
            
    #print(data) 확인
    cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", data)  # 문자형 날짜를 날짜형으로 변환(위 과정 축약)
    
conn.commit()
conn.close()

In [40]:

conn = MySQLdb.connect(**config)
cur = conn.cursor() 

sql = "select * from suppliers"
cur.execute(sql) 

for data in cur:  
    print(data) 
    
conn.close()

('A', '001-1001', '2341', 500000.0, datetime.date(2014, 1, 20))
('A', '001-1002', '2341', 500000.0, datetime.date(2014, 1, 20))
('A', '001-1003', '5467', 750000.0, datetime.date(2014, 1, 20))
('A', '001-1004', '5467', 750000.0, datetime.date(2014, 1, 20))
('B', '001-1005', '7009', 250000.0, datetime.date(2018, 1, 30))
('B', '001-1006', '7009', 250000.0, datetime.date(2018, 1, 30))
('B', '001-1007', '6650', 125000.0, datetime.date(2017, 2, 3))
('B', '001-1008', '6650', 125000.0, datetime.date(2017, 2, 3))
('C', '001-1009', '3321', 615000.0, datetime.date(2017, 2, 3))
('C', '001-1010', '3321', 615000.0, datetime.date(2017, 2, 10))
('C', '001-1011', '3321', 615000.0, datetime.date(2017, 2, 17))
('C', '001-1012', '3321', 615000.0, datetime.date(2017, 2, 24))


In [44]:
conn = MySQLdb.connect(**config)
cur = conn.cursor() 

cur.execute("select * from suppliers where cost>500000")
file = csv.writer(open("data/output.csv", "w"), delimiter=",")  # writer(): csv로 저장하기 위한 함수
                                                                # delimiter 구분 기호 결정
                                                                # DB 에서 가져오는 즉시 file로 저장

header = ["Supplier Name", "Invoice Number", "Part Number", "Cost", "Purchase Date"]
file.writerow(header)  # header 통해 제목만 저장

for row in cur:
    file.writerow(row)

conn.close()

---

# SQLite 연결

In [2]:
import sqlite3  # 내장되어 있어 설치x. import 통해 불러옴
print(sqlite3.sqlite_version)  

3.29.0


In [52]:
conn = sqlite3.connect("data/mysqlite.db")  # sqlite=계정x. connect(연결할 db): 파일 기반이기 때문에 없으면 자동으로 db 생성
                                            # DB 접속
print(conn)

cur = conn.cursor()  # 접속한 DB 사용 위해 cursor() 통한 객체 생성
print(cur)

conn.close()

### SELECT 관련

In [3]:

conn = sqlite3.connect("data/mysqlite.db") 
cur = conn.cursor()

sql = "select * from member"
cur.execute(sql)

for row in cur:
    print(row)

conn.close()

(1, '홍길동', '2019-1-1')
(2, '임꺽정', '2017-9-4')


In [3]:

import csv

file = csv.reader(open("data/input.csv", "r"), delimiter = ",")
header = next(file)
    # next 선언 했기 때문에 file의 첫 줄은 데이터로 분류되지 않고 제목으로 분류
    # next 선언 전: 0번째=supplier name / next 선언 후: 1번째=A
print(header)

conn = sqlite3.connect("data/sqlite_test.db") 
cur = conn.cursor()

for row in file:  # input.csv에 있는 데이터 개수만큼 반복
    data = []
    for col in range(len(header)):  # 컬럼 5개를 모아서 data 리스트에 저장
        data.append(row[col])  
        
    cur.execute("insert into suppliers values(?, ?, ?, ?, ?)", data)  # MySQL values(%s, %s,..), SQLite values(?, ?,..)
    
conn.commit()
conn.close()

['Supplier Name', 'Invoice Number', 'Part Number', 'Cost', 'Purchase Date']


In [ ]:
for row in file:
    # 1. row = A, 001, 1001, 2341, 500000, 2014-01-20
    # 5. file 변수의 input.csv 데이터 개수만큼 반복
   
    data = []
    for col in range(len(header)):
        # 2. col = 0번째
        data.append(row[col])
            # 3-1. data 변수에 추가: row의 0번째 데이터를
            # 3-2. 반복. row의 1번째 데이터를 data 변수에 추가~ 5번째 데이터까지
            # 3-3. 5번째 데이터까지 끝나면 반복 종료
    cur.execute("insert into suppliers values(?, ?, ?, ?, ?)", data)
    # 4. data에 저장한 데이터들을 양식에 맞춰 suppliers 테이블의 값에 넣는다
    


In [5]:
conn = sqlite3.connect("data/sqlite_test.db") 
cur = conn.cursor()

sql = "select * from suppliers"
cur.execute(sql)

for row in cur:
    print(row)

conn.close()

('A', '001-1001', '2341', 500000.0, '2014-01-20')
('A', '001-1002', '2341', 500000.0, '2014-01-20')
('A', '001-1003', '5467', 750000.0, '2014-01-20')
('A', '001-1004', '5467', 750000.0, '2014-01-20')
('B', '001-1005', '7009', 250000.0, '2018-01-30')
('B', '001-1006', '7009', 250000.0, '2018-01-30')
('B', '001-1007', '6650', 125000.0, '2017-02-03')
('B', '001-1008', '6650', 125000.0, '2017-02-03')
('C', '001-1009', '3321', 615000.0, '2017-02-03')
('C', '001-1010', '3321', 615000.0, '2017-02-10')
('C', '001-1011', '3321', 615000.0, '2017-02-17')
('C', '001-1012', '3321', 615000.0, '2017-02-24')


In [10]:
# 일부 데이터 조회 (메서드 사용)

conn = sqlite3.connect("data/sqlite_test.db") 
cur = conn.cursor()

sql = "select * from suppliers"
cur.execute(sql)

rows = cur.fetchmany(2)  # cur객체 내 전체 데이터 중 2개의 레코드만 가져온다
                         # fetchmany(): 원하는 개수만큼 레코드를 가져온다
                         # MySQL = select * from suppliers limit 2
for row in rows:
    print(row)
        
rows1 = cur.fetchone()  # fetchone(): 하나의 레코드만 가져온다(인자 필요x)
print(rows1)
print(rows1[0], rows1[1])
print("-" * 30)

conn.close()

('A', '001-1003', '5467', 750000.0, '2014-01-20')
A 001-1003
------------------------------
('A', '001-1001', '2341', 500000.0, '2014-01-20')
('A', '001-1002', '2341', 500000.0, '2014-01-20')


In [ ]:
'''
1. mysql의 scott_emp 테이블을 emp.csv로 저장

2. scott_emp 테이블 이용해서 아래와 같은 프로그램 작성
    1) 연결할 db 종류 선택:
        1. mysql
        2. sqlite
        
    2) 연결할 주소와 계정 입력(mysql):
        주소: xxx.xxx.xxx.xxx
        아이디: xxxx
        패스워드: xxxx
        
    3) 메뉴 선택:
        1. 조회
        2. 입력
        3. 수정
        4. 삭제
'''

In [1]:
import MySQLdb
config={"host":"127.0.0.1", "user":"root", "password":"1111", "database":"testdb"}  # testdb 접속.
conn = MySQLdb.connect(**config)

Database = input("연결할 db 종류 선택(1.MySQL, 2.SQLite): ")

if Database == 1:
    host = input("연결할 주소: ")
    user = input("아이디 입력: ")
    password = input("패스워드 입력: ")

            
        
    

IndentationError: unexpected indent (<ipython-input-1-699ee3f27df4>, line 12)

---

# MongoDB

In [1]:
# pip install pymongo
!pip install pymongo

In [4]:
#import pymongo
from pymongo import MongoClient

In [6]:
#conn = MongoClient("127.0.0.1", 27017)  # 27017 = 포트번호
conn = MongoClient("mongodb://127.0.0.1:27017")  # 서버 연결
print(conn)

conn.close()

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)


In [10]:
conn = MongoClient("mongodb://127.0.0.1:27017")
#db = conn.testdb  # testdb와 연결
#db = conn["testdb"]
db = conn.get_database("testdb")  # get_database() 메서드 이용한 testdb연결
print(db)

conn.close()

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'testdb')


In [17]:
conn = MongoClient("mongodb://127.0.0.1:27017")  # 서버연결
db = conn.get_database("testdb")  # db연결
#collect = db.articles  # 컬렉션 연결
#collect = db["articles"]
collect = db.get_collection("articles")
print(collect)

conn.close()

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'testdb'), 'articles')


In [18]:
# insert_one() 하나씩 레코드 입력

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

data = {"author":"Mike", "text":"my first blog post", "tag":["mongodb", "python"]}
collect.insert_one(data)  # people 컬렉션에 insert_one() 이용하여 data변수의 값 삽입

conn.close()

In [20]:
# _id 값까지 출력 : inserted_id

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

data = {"author":"Mike", "text":"my first blog post", "tag":["mongodb", "python"]}
post_id = collect.insert_one(data).inserted_id  # inserted_id : 입력될 때 생성되는 _id값을 리턴받는다
print(post_id)

conn.close()

5d6391f300c973b68c90eda2


In [25]:
# 현재 collect 안에 들어있는 document 개수 확인 : count()
# 연속된 값 입력 경우 : 반복문 사용

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

print(collect.count())

collect.insert_many([{"number":i} for i in range(10)])  # 여러 개의 레코드를 한번에 입력
print(collect.count())

conn.close()

9
19


C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  import sys
C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  # Remove the CWD from sys.path while we load stuff.


In [27]:
# 입력하려는 값 다른 경우

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

data = list()
data.append({"name":"aron", "age":20})
data.append({"name":"ton", "age":21})
data.append({"name":"jane", "age":22})
data.append({"name":"jerry", "age":23})

collect.insert_many(data)

conn.close()

In [28]:
# 반복문 통해 컬렉션 전체 출력

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

for i in collect.find():
    print(i)

conn.close()

{'_id': ObjectId('5d63591b5550541136db3fc3'), 'name': 'Abet', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5d63591b5550541136db3fc4'), 'name': 'Betty', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5d63591b5550541136db3fc5'), 'name': 'Charlie', 'age': 23.0, 'skills': ['mongodb', 'python']}
{'_id': ObjectId('5d63591b5550541136db3fc6'), 'name': 'David', 'age': 23.0, 'score': 20.0}
{'_id': ObjectId('5d637952957a62e0e1e892cb'), 'name': 'Elly', 'age': 17.0, 'score': 10.0}
{'_id': ObjectId('5d6379a9957a62e0e1e892d4'), 'name': 'ddd', 'age': 17.0, 'score': 10.0}
{'_id': ObjectId('5d638f1c00c973b68c90ed9e'), 'author': 'Mike', 'text': 'my first blog post', 'tag': ['mongodb', 'python']}
{'_id': ObjectId('5d63919500c973b68c90eda0'), 'author': 'Mike', 'text': 'my first blog post', 'tag': ['mongodb', 'python']}
{'_id': ObjectId('5d6391f300c973b68c90eda2'), 'author': 'Mike', 'text': 'my first blog post', 'tag': ['mongodb', 'python']}
{'_id': ObjectId('5d63936a00c973b68c90eda5'), 'number': 0}
{'_id':

In [30]:
# 특정 데이터만

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

print(collect.find_one())  # 맨 위 하나만 출력
print(collect.find_one({"author":"Mike"}))  # 조건 통해 하나만 출력

conn.close()

{'_id': ObjectId('5d63591b5550541136db3fc3'), 'name': 'Abet', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5d638f1c00c973b68c90ed9e'), 'author': 'Mike', 'text': 'my first blog post', 'tag': ['mongodb', 'python']}


In [31]:
# 수정 : update_one(), update_many

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.get_database("testdb")
collect = db.get_collection("people")

collect.update_one({"author":"Mike"}, {"$set":{"text":"my second blog post"}})

conn.close()